In [1]:
%pwd

'C:\\Users\\ASUS'

<a href="https://colab.research.google.com/github/jylee2930/DB_Basic/blob/main/Yolov5_AI_Specialist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Yolo5 학습하기
##1. 구글 드라이브 연동

In [2]:
%cd C:\Users\ASUS\Desktop\AI

C:\Users\ASUS\Desktop\AI


In [3]:
%pwd

'C:\\Users\\ASUS\\Desktop\\AI'

# 2. Yolov5 설치

In [4]:
# 기존에 설치를 완료한 경우에는 해당 경로로 이동만 하면 됩니다.
%cd C:\Users\ASUS\Desktop\AI\yolov5\yolov5

C:\Users\ASUS\Desktop\AI\yolov5\yolov5


In [4]:
!git clone https://github.com/ultralytics/yolov5
%cd C:\Users\ASUS\Desktop\AI\yolov5
%pip install -qr requirements.txt

C:\Users\ASUS\Desktop\AI\yolov5


Cloning into 'yolov5'...


Note: you may need to restart the kernel to use updated packages.


In [22]:
!pip install Pillow==10.3

#

# 3. 이미지 파일 관리

In [13]:
##검증 데이터 만들기
import os
import shutil
from sklearn.model_selection import train_test_split

def create_validation_set(train_path, val_path, split_ratio=0.3):
    """
    Train 데이터의 일부를 Val로 이동
    """
    # 필요한 디렉토리 생성
    os.makedirs(os.path.join(val_path, 'images'), exist_ok=True)
    os.makedirs(os.path.join(val_path, 'labels'), exist_ok=True)

    # Train 이미지 리스트 가져오기
    train_images = os.listdir(os.path.join(train_path, 'images'))
    train_images = [f for f in train_images if f.endswith(('.jpg', '.jpeg', '.png'))]

    # Train/Val 분할
    _, val_images = train_test_split(train_images,
                                   test_size=split_ratio,
                                   random_state=42)

    # Val로 파일 복사
    for image_file in val_images:
        # 이미지 복사
        src_image = os.path.join(train_path, 'images', image_file)
        dst_image = os.path.join(val_path, 'images', image_file)
        shutil.copy2(src_image, dst_image)

        # 라벨 파일 복사
        label_file = os.path.splitext(image_file)[0] + '.txt'
        src_label = os.path.join(train_path, 'labels', label_file)
        dst_label = os.path.join(val_path, 'labels', label_file)
        if os.path.exists(src_label):
            shutil.copy2(src_label, dst_label)

    print(f"Created validation set with {len(val_images)} images")

# 실행
train_path = 'C:\\Users\\ASUS\\Desktop\\AI\\yolov5\\yolov5\\Train'
val_path = 'C:\\Users\\ASUS\\Desktop\\AI\\yolov5\\yolov5\\Val'

create_validation_set(train_path, val_path)

Created validation set with 806 images


In [14]:
def check_dataset():
    train_path = 'C:\\Users\\ASUS\\Desktop\\AI\\yolov5\\yolov5\\Train'
    val_path = 'C:\\Users\\ASUS\\Desktop\\AI\\yolov5\\yolov5\\Val'

    # Train 데이터 확인
    train_images = len(os.listdir(os.path.join(train_path, 'images')))
    train_labels = len(os.listdir(os.path.join(train_path, 'labels')))

    # Val 데이터 확인
    val_images = len(os.listdir(os.path.join(val_path, 'images')))
    val_labels = len(os.listdir(os.path.join(val_path, 'labels')))

    print("Dataset status:")
    print(f"Train - Images: {train_images}, Labels: {train_labels}")
    print(f"Val - Images: {val_images}, Labels: {val_labels}")

# 데이터셋 상태 확인
check_dataset()

Dataset status:
Train - Images: 2684, Labels: 2684
Val - Images: 806, Labels: 806


# 4. 학습시작

In [15]:
import torch
import os
from IPython.display import Image, clear_output  # to display images

In [16]:
%pwd

'C:\\Users\\ASUS\\Desktop\\AI\\yolov5\\yolov5'

In [17]:

import numpy as np
import tensorflow as tf
import os
from PIL import Image
from tensorflow.python.eager.context import eager_mode

def _preproc(image, output_height=512, output_width=512, resize_side=512):
    ''' imagenet-standard: aspect-preserving resize to 256px smaller-side, then central-crop to 224px'''
    with eager_mode():
        h, w = image.shape[0], image.shape[1]
        scale = tf.cond(tf.less(h, w), lambda: resize_side / h, lambda: resize_side / w)
        resized_image = tf.compat.v1.image.resize_bilinear(tf.expand_dims(image, 0), [int(h*scale), int(w*scale)])
        cropped_image = tf.compat.v1.image.resize_with_crop_or_pad(resized_image, output_height, output_width)
        return tf.squeeze(cropped_image)

def Create_npy(imagespath, imgsize, ext) :
    images_list = [img_name for img_name in os.listdir(imagespath) if
                os.path.splitext(img_name)[1].lower() == '.'+ext.lower()]
    calib_dataset = np.zeros((len(images_list), imgsize, imgsize, 3), dtype=np.float32)

    for idx, img_name in enumerate(sorted(images_list)):
        img_path = os.path.join(imagespath, img_name)
        try:
            # 파일 크기가 정상적인지 확인
            if os.path.getsize(img_path) == 0:
                print(f"Error: {img_path} is empty.")
                continue

            img = Image.open(img_path)
            img = img.convert("RGB")  # RGBA 이미지 등 다른 형식이 있을 경우 강제로 RGB로 변환
            img_np = np.array(img)

            img_preproc = _preproc(img_np, imgsize, imgsize, imgsize)
            calib_dataset[idx,:,:,:] = img_preproc.numpy().astype(np.uint8)
            print(f"Processed image {img_path}")

        except Exception as e:
            print(f"Error processing image {img_path}: {e}")

    np.save('calib_set.npy', calib_dataset)


In [18]:
Create_npy('C:\\Users\\ASUS\\Desktop\\AI\\yolov5\\yolov5\\Train\\images', 512, 'jpg')

In [28]:
python "C:\\Users\\ASUS\\Desktop\\AI\\yolov5\\yolov5\\train.py"  --img 512 --batch 16 --epochs 300 --data C:\Users\ASUS\Desktop\AI\yolov5\yolov5\data.yaml --weights yolov5n.pt --cache

SyntaxError: invalid syntax (2054092675.py, line 1)

In [19]:
from torch.utils.tensorboard import SummaryWriter

writer = SummaryWriter("runs/experiment1")  # logdir에 이벤트 파일을 저장
# 임의의 예제 데이터를 추가 (예: 손실 값 기록)
for i in range(100):
    writer.add_scalar("Loss/train", 0.1 * i, i)
writer.close()


In [20]:
%load_ext tensorboard
%tensorboard --logdir runs


In [21]:
%load_ext tensorboard
%tensorboard --logdir "C:/Users/ASUS/Desktop/AI/yolov5/yolov5/runs/train/exp23"

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [22]:
import glob
from IPython.display import Image, displa

for imageName in glob.glob('C:/Users/ASUS/Desktop/AI/yolov5/yolov5/runs/train/exp23*.jpg')[:10]:
    display(Image(filename=imageName))
    print("\n")

ImportError: cannot import name 'displa' from 'IPython.display' (C:\Users\ASUS\anaconda3\Lib\site-packages\IPython\display.py)

In [2]:
!python detect.py --weights C:\\Users\\ASUS\\Desktop\\AI\\yolov5\\yolov5\\runs\\train\\exp23\\weights\\best.pt --img 512 --conf 0.1 --source C:\\Users\\ASUS\\Desktop\\AI\\yolov5\\yolov5\\images

python: can't open file 'C:\\Users\\ASUS\\detect.py': [Errno 2] No such file or directory
